In [1]:
import pandas as pd
import numpy as np

actual = pd.read_csv('Italy_data.csv')
forecast_prices = pd.read_csv("predictions_prices.csv")
forecast_demand = pd.read_csv("predictions_consumption.csv")

# Ensure all data is in the same format
actual['timestamps'] = pd.to_datetime(actual['timestamps'])
forecast_prices['timestamps'] = pd.to_datetime(forecast_prices['timestamps'])
forecast_demand['timestamps'] = pd.to_datetime(forecast_demand['timestamps'])

actual.sort_values('timestamps', inplace=True)
forecast_prices.sort_values('timestamps', inplace=True)
forecast_demand.sort_values('timestamps', inplace=True)



/var/folders/t4/gd0ch0vd2d9gfydnl1mwjj7c0000gn/T/ipykernel_27592/3061943113.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  forecast_prices['timestamps'] = pd.to_datetime(forecast_prices['timestamps'])
/var/folders/t4/gd0ch0vd2d9gfydnl1mwjj7c0000gn/T/ipykernel_27592/3061943113.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  forecast_demand['timestamps'] = pd.to_datetime(forecast_demand['timestamps'])


In [2]:
forecast_prices.tail()

,timestamps,prices,RF_pred,LSTM_pred,prices + noise
17515,2019-12-31 19:00:00,56.48,74.521129,82.553541,54.310326
17516,2019-12-31 20:00:00,53.24,73.560176,79.519272,52.782918
17517,2019-12-31 21:00:00,50.68,69.016322,77.329024,48.470322
17518,2019-12-31 22:00:00,48.61,65.635389,72.623072,48.921003
17519,2019-12-31 23:00:00,42.30,57.978904,71.313366,43.188443


In [3]:
forecast_demand.tail()

,timestamps,consumption,RF_pred,LSTM_pred,consumption + noise
17515,2019-12-31 19:00:00,34.89,44.246491,45.659468,33.258347
17516,2019-12-31 20:00:00,32.54,40.480962,44.297204,32.851411
17517,2019-12-31 21:00:00,30.03,37.848983,41.369403,29.049828
17518,2019-12-31 22:00:00,27.64,34.336359,38.391652,28.091693
17519,2019-12-31 23:00:00,25.40,28.446134,34.658032,24.689197


In [4]:
actual.head()

,timestamps,prices,consumption,fossil_generation,wind_generation,month
0,2022-01-01 00:00:00,170.28,22.40,11.05,6.81,winter
1,2022-01-01 01:00:00,155.72,20.79,10.36,6.36,winter
2,2022-01-01 02:00:00,147.09,19.85,9.60,6.12,winter
3,2022-01-01 03:00:00,91.00,19.12,9.16,5.94,winter
4,2022-01-01 04:00:00,104.00,19.08,8.43,6.02,winter


In [5]:
data_actual =  pd.merge(forecast_prices[['timestamps', 'prices']], forecast_demand[['timestamps', 'consumption']], on='timestamps', how='inner')
# data_actual.rename(columns={'actual_pred_price': 'prices', 'actual_pred_demand': 'consumption'}, inplace=True)
data_actual.sort_values('timestamps', inplace=True)
data_actual.dropna(inplace=True)
# find out the duplicate timestamps in the data_actual dataframe
duplicates = data_actual[data_actual.duplicated('timestamps', keep=False)]

# remove these duplicates keeping the first occurrence
data_actual = data_actual.drop_duplicates('timestamps', keep='first')
data_actual.reset_index(drop=True, inplace=True)
data_actual

,timestamps,prices,consumption
0,2018-01-01 00:00:00,45.82,22.98
1,2018-01-01 01:00:00,44.16,21.81
2,2018-01-01 02:00:00,42.24,20.76
3,2018-01-01 03:00:00,39.29,19.97
4,2018-01-01 04:00:00,36.00,19.49
...,...,...,...
17513,2019-12-31 19:00:00,56.48,34.89
17514,2019-12-31 20:00:00,53.24,32.54
17515,2019-12-31 21:00:00,50.68,30.03
17516,2019-12-31 22:00:00,48.61,27.64


In [6]:
data_noise =  pd.merge(forecast_prices[['timestamps', 'prices + noise']], forecast_demand[['timestamps', 'consumption + noise']], on='timestamps', how='inner')
data_noise.rename(columns={'prices + noise': 'prices', 'consumption + noise': 'consumption'}, inplace=True)
data_noise.sort_values('timestamps', inplace=True)
data_noise.dropna(inplace=True)
# find out the duplicate timestamps in the data_noise dataframe
duplicates = data_noise[data_noise.duplicated('timestamps', keep=False)]

# remove these duplicates keeping the first occurrence
data_noise = data_noise.drop_duplicates('timestamps', keep='first')
data_noise.reset_index(drop=True, inplace=True)
data_noise

,timestamps,prices,consumption
0,2018-01-01 00:00:00,46.073502,24.515765
1,2018-01-01 01:00:00,43.730295,22.733190
2,2018-01-01 02:00:00,42.851569,19.517337
3,2018-01-01 03:00:00,37.710871,20.132493
4,2018-01-01 04:00:00,36.055618,19.239896
...,...,...,...
17513,2019-12-31 19:00:00,54.310326,33.258347
17514,2019-12-31 20:00:00,52.782918,32.851411
17515,2019-12-31 21:00:00,48.470322,29.049828
17516,2019-12-31 22:00:00,48.921003,28.091693


In [7]:
data_LSTM =  pd.merge(forecast_prices[['timestamps', 'LSTM_pred']], forecast_demand[['timestamps', 'LSTM_pred']], on='timestamps', suffixes=('_price', '_demand'))
data_LSTM.rename(columns={'LSTM_pred_price': 'prices', 'LSTM_pred_demand': 'consumption'}, inplace=True)
data_LSTM.sort_values('timestamps', inplace=True)
data_LSTM.dropna(inplace=True)
# find out the duplicate timestamps in the data_LSTM dataframe
duplicates = data_LSTM[data_LSTM.duplicated('timestamps', keep=False)]

# remove these duplicates keeping the first occurrence
data_LSTM = data_LSTM.drop_duplicates('timestamps', keep='first')
data_LSTM.reset_index(drop=True, inplace=True)
data_LSTM

,timestamps,prices,consumption
0,2018-01-01 00:00:00,76.913203,35.051603
1,2018-01-01 01:00:00,76.245376,36.221259
2,2018-01-01 02:00:00,73.325385,35.518524
3,2018-01-01 03:00:00,69.120745,33.928986
4,2018-01-01 04:00:00,63.816008,32.341698
...,...,...,...
17513,2019-12-31 19:00:00,82.553541,45.659468
17514,2019-12-31 20:00:00,79.519272,44.297204
17515,2019-12-31 21:00:00,77.329024,41.369403
17516,2019-12-31 22:00:00,72.623072,38.391652


In [8]:
data_rf =  pd.merge(forecast_prices[['timestamps', 'RF_pred']], forecast_demand[['timestamps', 'RF_pred']], on='timestamps', suffixes=('_price', '_demand'))
data_rf.rename(columns={'RF_pred_price': 'prices', 'RF_pred_demand': 'consumption'}, inplace=True)
data_rf.sort_values('timestamps', inplace=True)
data_rf.dropna(inplace=True)
# find out the duplicate timestamps in the data_rf dataframe
duplicates = data_rf[data_rf.duplicated('timestamps', keep=False)]

# remove these duplicates keeping the first occurrence
data_rf = data_rf.drop_duplicates('timestamps', keep='first')
data_rf.reset_index(drop=True, inplace=True)
data_rf

,timestamps,prices,consumption
0,2018-01-01 00:00:00,47.318961,24.956646
1,2018-01-01 01:00:00,41.784323,23.929971
2,2018-01-01 02:00:00,37.492516,22.429782
3,2018-01-01 03:00:00,36.728515,21.715821
4,2018-01-01 04:00:00,36.248728,21.876235
...,...,...,...
17513,2019-12-31 19:00:00,74.521129,44.246491
17514,2019-12-31 20:00:00,73.560176,40.480962
17515,2019-12-31 21:00:00,69.016322,37.848983
17516,2019-12-31 22:00:00,65.635389,34.336359


In [9]:
data_actual.to_csv('Italy_data_actual.csv', index=False)
data_noise.to_csv('Italy_data_forecast_NOISE.csv', index=False)
data_LSTM.to_csv('Italy_data_forecast_LSTM.csv', index=False)
data_rf.to_csv('Italy_data_forecast_RF.csv', index=False)